In [ ]:
import sys
sys.path.append('sources')
import traceback
import os
import json

# Choose the test
All the test available are in folder 'CPIs'

In [ ]:
process_name = "test5" #choice-task-init-reverse

## Load File 

In [ ]:
cpi_file_path = f'CPIs/{process_name}.cpi'

print(f"Loading CPI file: {cpi_file_path}")

try:
	with open(cpi_file_path, 'r') as f:
		cpi_dict = json.load(f)

	print("✓ CPI file loaded successfully!")
	print(f"Root region type: {cpi_dict['type']}")
	print(f"Root region ID: {cpi_dict['id']}")

	# Pretty print the CPI structure
	print("\nCPI Structure:")
	print("=" * 50)
	print(json.dumps(cpi_dict, indent=2))

except FileNotFoundError:
	print(f"❌ File not found: {cpi_file_path}")
	print("Available files in CPIs directory:")
	try:
		for f in os.listdir('CPIs'):
			if f.endswith('.cpi'):
				print(f"  - {f}")
	except:
		print("  Could not list CPIs directory")
except Exception as e:
	print(f"❌ Error loading CPI file: {e}")
	traceback.print_exc()

# FROM CPI TO SPIN

In [ ]:
from cpi_to_mdp.cpitospin import analyze_cpi_structure, CPIToSPINConverter

print("\nCPI Structure Analysis:")
print("=" * 50)
if 'cpi_dict' in locals():
	analyze_cpi_structure(cpi_dict)

print("Converting CPI to SPIN...")
print("=" * 50)

try:
	converter = CPIToSPINConverter()
	spin_model = converter.convert_cpi_to_spin(cpi_dict)

	print("✓ Conversion successful!")
	print("\nSPIN Model Summary:")
	print("-" * 30)
	spin_model.print_model_summary()

except Exception as e:
	print(f"❌ Conversion failed: {e}")
	traceback.print_exc()

## Process Visualization

The CPI dictionary can be visualized as a directed graph to better understand its structure. In this visualization:

- **Task nodes** show duration and impact values (cost, time, quality)
- **Nature nodes** display their probability values (e.g., "p=0.7")
- **Sequence nodes** connect components with "head" and "tail" edges
- **Parallel nodes** show concurrent branches with "first" and "second" edges
- **Choice nodes** represent decision points with "true" and "false" branches

Each node type is represented as a box, with edges showing the relationships between components. This hierarchical representation helps understand the process flow and decision points in the system.

In [ ]:
from cpi_to_mdp.cpitospin import create_cpi_visualization, create_spin_visualization

print("\nCreating visualizations...")
print("=" * 50)

try:
	# Create CPI visualization
	cpi_viz = create_cpi_visualization(cpi_dict, "CPI: Loop Example")
	print("✓ CPI visualization created")

	# Create SPIN visualization
	spin_viz = create_spin_visualization(spin_model, "SPIN: Loop Example")
	print("✓ SPIN visualization created")

	# Display visualizations (if in Jupyter)
	try:
		print("\nCPI Structure Visualization:")
		display(cpi_viz)

		print("\nSPIN Model Visualization:")
		display(spin_viz)
	except NameError:
		# Not in Jupyter, save to files instead
		print("Saving visualizations to files...")

		# Save CPI visualization
		cpi_output = process_name + '_cpi'
		cpi_viz.render(cpi_output, cleanup=True)
		print(f"CPI visualization saved to: {cpi_output}.png")

		# Save SPIN visualization
		spin_output = process_name + '_spin'
		spin_viz.render(spin_output, cleanup=True)
		print(f"SPIN visualization saved to: {spin_output}.png")

except Exception as e:
	print(f"❌ Visualization failed: {e}")
	traceback.print_exc()


# FROM SPIN TO PRISM

In [ ]:
print("\nGenerating PRISM model...")
print("=" * 50)

try:
	prism_model = spin_model.generate_prism_model()

	print("✓ PRISM model generated successfully!")

	# Show model statistics
	lines = prism_model.split('\n')
	words = prism_model.split()

	print(f"Model statistics:")
	print(f"  Lines: {len(lines)}")
	print(f"  Words: {len(words)}")
	print(f"  Characters: {len(prism_model)}")

	# Show first 20 lines
	print(f"\nFirst 20 lines of PRISM model:")
	print("-" * 40)
	for i, line in enumerate(lines[:20]):
		print(f"{i + 1:2d}: {line}")

	if len(lines) > 20:
		print(f"... ({len(lines) - 20} more lines)")

except Exception as e:
	print(f"❌ PRISM generation failed: {e}")
	traceback.print_exc()

print("\nSaving PRISM model...")
print("=" * 50)

try:
	output_file =  "models/" + process_name + '.nm'
	with open(output_file, 'w') as f:
		f.write(prism_model)

	print(f"✓ PRISM model saved to: {output_file}")

	# Verify file was created
	if os.path.exists(output_file):
		file_size = os.path.getsize(output_file)
		print(f"File size: {file_size} bytes")
	else:
		print("❌ Warning: File was not created")

except Exception as e:
	print(f"❌ Save failed: {e}")
	traceback.print_exc()

## RUN PRISM ANALYSIS

In [ ]:
!prism

In [ ]:
from env import PRISM_PATH
from prism import run_prism_analysis

prism_path = None # If PRISM_PATH is set in the environment
#prism_path = PRISM_PATH
run_prism_analysis(process_name, create_mdp=True)


### Read output

In [ ]:
from graphviz import Source

dot_filename = f"models/{process_name}.dot"

with open(dot_filename, 'r', encoding='utf-8') as f: #{process_name}/{process_name}.dot
    dot_content = f.read()
    Source(dot_content).render(filename=f"models/{process_name}", format='svg', cleanup=True)

states_filenames = f"models/{process_name}_states.csv"
transitions_filenames = f"models/{process_name}_trans.tra"

# FROM PRISM TO MDP

Since PRISM models are based on an extended form of MDPs, we now provide the compact version of the MDP generated by PRISM, which corresponds to the equivalent SPIN model. This enhances clarity and facilitates easier comparison between the two representations.

CASE 1: deterministic, 2 states directly connected

CASE 2: time passage 2 states connected with label="{{∅}}" fillcolor=lightsalmon

Case 3: choice: red transition with internal writing some choice I take (1+) and saying true/false --> written in the next state

case 4 natural: as choice but green transition and label on the probability arcs

Case 5: nature and contemporary choice: double transition first choice and then nature

In [ ]:
from graphviz import Source
from collections import defaultdict
import pandas as pd
import re

def get_compact_states(states: pd.DataFrame, transitions: pd.DataFrame):
    states['STAGE'] = states['(STAGE'].apply(
        lambda x: x.split(':')[-1].split('(')[-1] if isinstance(x, str) else x
    )
    states.drop(columns=['(STAGE'], inplace=True)

    paren_cols = [col for col in states.columns if col.endswith(')')]
    
    for col in paren_cols:
        states[col.replace(')', '')] = states[col].apply(
            lambda x: x[-2] if isinstance(x, str) and len(x) >= 2 else x
        )
        states.drop(columns=[col], inplace=True)

    states = states.apply(pd.to_numeric)

    update_cols = [col for col in states.columns if col.endswith('_update')]
    state_cols = [col for col in states.columns if col.endswith('_state')]
    value_cols = [col for col in states.columns if col.endswith('_value')]  

    stage_mask = states['STAGE'].isin([0])    
    update_mask = (states[update_cols] == 0).all(axis=1)    
    state_mask = (states[state_cols] == 0).all(axis=1)
    new_states_df = states[stage_mask & update_mask & state_mask]
    return new_states_df, set(new_states_df.index.astype(int).tolist()), value_cols

def prepare_label_states_dot(name:int =0, parts = {}):
    joined_parts = ''
    for k, v in parts.items():
        joined_parts += f'{k} → {v} \n'
    return f'{name} [label="{joined_parts}", style="filled", fillcolor="lightblue"];'
def format_label(row, value_cols):
    parts = {}
    '''
    {
        col.replace('_value', ''): [
                             int(row[col]), # valore temporale
                             1 if choice and true in col elif 0 if choice and false  else  None #mi segno se choice e in quale ramo
                             1 if nature and true in col else 0 #mi segno se nature e in quale ramo
                                    ]
        
    }
    '''
    for col in value_cols:
        if row[col] >= 0:
            parts[col.replace('_value', '')] = int(row[col])
    return prepare_label_states_dot(row.name, parts), parts

def create_trans_dict(transitions:pd.DataFrame, verbose = False):
    trans = transitions[0].str.split(expand=True)
    if verbose:
        print(trans.head())
        print(trans.shape)
    has_prob = trans.shape[1] >= 5
    
    # Extract relevant columns
    if has_prob:
        source_dest = trans[[0, 2, 3, 4]]
        source_dest.columns = ['source', 'destination', 'prob', 'label']
    else:
        source_dest = trans[[0, 2, 3]]
        source_dest.columns = ['source', 'destination', 'label']
    trans_dict = defaultdict(list)
    
    for _, row in source_dest.iterrows():
        source = int(row['source'])
        destination = int(row['destination'])
        prob = float(row['prob']) if has_prob else 1.0  # Default prob to 1.0 if missing
        label = [row['label'] ]if row['label'] else []
        trans_dict[source].append((destination, prob,label))
    
    return dict(trans_dict)

def find_next_state(src:int, trans_dict:dict, possible_targets: set, verbose = False):  
        res = []
        for i in range(len(trans_dict[src])):
            if src == trans_dict[src][i][0]:
                return [] # it's final 
            if trans_dict[src][i][0] in possible_targets:
                if verbose:
                    print(trans_dict[src][i][0], trans_dict[src][i][1])
                res.append(trans_dict[src][i]) # questo significa che src e target sono collegati direttamente 
            
        if len(res) > 0:
            return res
        for i in range(len(trans_dict[src])): 
            # continua a cercare ma la destinazione è diventata source
            l = trans_dict[src][i][2]
            for r in find_next_state( trans_dict[src][i][0], trans_dict, possible_targets):
                l.extend(r[2])
                res.append((r[0], round(r[1] * trans_dict[src][i][1], 3), l))            
        return res

def parse_rewards_file(filepath):
    rewards_dict = defaultdict(dict)
    current_impact = None

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            
            if line.startswith('rewards'):
                match = re.search(r'rewards\s+"([^"]+)"', line)
                if match:
                    current_impact = match.group(1)

            elif line == 'endrewards':
                current_impact = None

            elif current_impact:
                match = re.match(r'\[([^\]]+)\]\s+true\s*:\s*([0-9.eE+-]+);', line)
                
                if match:
                    task_name = match.group(1)
                    value = float(match.group(2))
                    rewards_dict[task_name][current_impact] = value

    return dict(rewards_dict)

def format_label_impacts_impactslabel(label: list, rewards_dict: dict):
    l = []
    impact_sums = {}  # Accumulate sums for each impact
    
    for col in label:
        l.append(f"{col.replace('fire_', '')}")
        for impact, value in rewards_dict[col].items():
            print(label)
            if impact in impact_sums:
                impact_sums[impact] += value
            else:
                impact_sums[impact] = value

    # Format summed impacts as "impact_name:total_value"
    lab_impacts = [f'{impact}:{value}' for impact, value in impact_sums.items()]
    return '\n'.join(l), '\n'.join(lab_impacts)
    
def create_states_mdp(states:pd.DataFrame, transitions:pd.DataFrame, process_name = 'test0', save =False):
    
    new_states_df, possible_targets, value_cols = get_compact_states(states, transitions)
    trans_dict = create_trans_dict(transitions)
    rewards_dict = parse_rewards_file(f"models/{process_name}.nm")
    lines = ['digraph LTS {', 'node [label="", shape="box"];']
    # print(new_states_df)
    idx_trans = max(possible_targets)+1 
    rows= dict(new_states_df.iterrows())
    for idx, row in rows.items():
        place_name_dot, place_name = format_label(row, value_cols)
        lines.append(place_name_dot) # add node states
        targets = find_next_state(idx, trans_dict, possible_targets)        
        if len(targets) == 1:
            # case 1 or case 2
            next_idx_target, _, label = targets[0]
            _, place_name_next = format_label(rows[next_idx_target], value_cols)
            if label: # add tasks and impacts 
                l , lab_impacts = format_label_impacts_impactslabel(label, rewards_dict)
                label_node_id = f'label_{idx_trans}'
                lines.append(f'{label_node_id} [label="{l} \\n {lab_impacts}", shape="box", style="filled,dashed", fillcolor="lightgrey"];')
                lines.append(f'{idx} -> {label_node_id} [style="dotted"];')
            print(place_name, 'place name', )
            print(place_name_next, 'place name next', )
            def is_passing_time(place_name, place_name_next):
                for k, v in place_name_next.items():
                    if k in place_name.keys():
                        if place_name[k] < v:
                            return True # if passing time 
                return False # if not passing time
            if is_passing_time(place_name, place_name_next):
                # Case 2
                lines.append(f'{idx_trans} [label="{{∅}}" , style="filled", fillcolor="lightsalmon", shape="circle"];')      
                lines.append(f'{idx} -> {idx_trans};')
                lines.append(f'{idx_trans} -> {next_idx_target}; ')
            else:
                # Case 1              
                lines.append(f'{idx} -> {next_idx_target};')
        else: 
            pass
            # for t in targets:
            #     next_idx, probability, label = t
            #     prob_label = f'[label = ""]'     
            #     _, place_name_next = format_label(next_row, value_cols)
            #     if 'choice' in place_name_next  and probability == 1.0 and 'nature' not in place_name_next:
            #         # Case 3: only choices
            #         lines.append(f'{idx_trans} [label="{{{place_name_next}}}" , style="filled", fillcolor="lightcoral", shape="circle" ];')
            #         lines.append(f'{idx} -> {idx_trans};')
            #         lines.append(f'{idx_trans} -> {next_idx} {prob_label}; ')
            #     elif 'nature' in place_name_next and probability < 1.0 and 'choice' not in place_name_next:
            #         # Case 4: only natures
            #         prob_label = f'[label = "{probability}"]'
            #         lines.append(f'{idx_trans} [label="{{{place_name_next}}}" , style="filled", fillcolor="lightgreen", shape="circle" ];')
            #         lines.append(f'{idx} -> {idx_trans};')
            #         lines.append(f'{idx_trans} -> {next_idx} {prob_label}; ')
            #     else:
            #         # Case 5: natures and choices
            #         idx_nature = f'{idx_trans*25}'
            #         lines.append(f'{idx_trans} [label="{{{place_name_next}}}" , style="filled", fillcolor="lightcoral", shape="circle" ];')
            #         lines.append(f'{idx_nature} [label="{{{place_name_next}}}" , style="filled", fillcolor="lightgreen", shape="circle" ];')
            #         lines.append(f'{idx} -> {idx_trans};')
            #         lines.append(f'{idx_trans} -> {idx_nature};')
            #         prob_label = f'[label = "{probability}"]'                    
                    # lines.append(f'{idx_nature} -> {next_idx} {prob_label}; ')

                    # idx -> idx_choice -> idx_nat -> idx_next
        idx_trans +=1
    lines.append('}')
    compress_dot = "\n".join(lines)
    if save:
        Source(compressed_dot).render(filename=f"models/{process_name}_cleaned", format='svg', cleanup=True)
    return compress_dot


states = pd.read_csv(states_filenames)
transitions = pd.read_csv(transitions_filenames, skiprows=1, header= None)

compressed_dot = create_states_mdp(states, transitions, process_name)

#print(compressed_dot)
Source(compressed_dot)

In [ ]:
print(compressed_dot)

In [ ]:
states

In [ ]:
transitions